In [1]:
import findspark
findspark.init()
from pyspark import SparkContext,SparkConf
#from pyspark.sql import SparkSession

In [2]:
conf = SparkConf().setAppName("PySpark01").setMaster("local[2]")
sc = SparkContext(conf=conf)


In [13]:
dir(sc)

['PACKAGE_EXTENSIONS',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accumulatorServer',
 '_active_spark_context',
 '_batchSize',
 '_callsite',
 '_checkpointFile',
 '_conf',
 '_dictToJavaMap',
 '_do_init',
 '_encryption_enabled',
 '_ensure_initialized',
 '_gateway',
 '_getJavaStorageLevel',
 '_initialize_context',
 '_javaAccumulator',
 '_jsc',
 '_jvm',
 '_lock',
 '_next_accum_id',
 '_pickled_broadcast_vars',
 '_python_includes',
 '_repr_html_',
 '_serialize_to_jvm',
 '_temp_dir',
 '_unbatched_serializer',
 'accumulator',
 'addFile',
 'addPyFile',
 'appName',
 'applicationId',
 'binaryFiles',
 'binaryRecords',
 'broadcas

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 3712)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 720, in __init__
    self.handle()
  File "C:\software\spark-2.3.3-bin-hadoop2.7\python\pyspark\accumulators.py", line 270, in handle
    poll(accum_updates)
  File "C:\software\spark-2.3.3-bin-hadoop2.7\python\pyspark\accumulators.py", line 242, in poll
    if func():
  File "C:\software\spark-2.3.3-bin-hadoop2.7\python\pyspark\accumulators.py", line 246, in accum_updat

In [10]:
#使用parallelize从现有集合中产生rdd
data = list(range(10))
rdd1 = sc.parallelize(data)

In [12]:
dir(rdd1)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_computeFractionForSampleSize',
 '_defaultReducePartitions',
 '_id',
 '_jrdd',
 '_jrdd_deserializer',
 '_memory_limit',
 '_pickled',
 '_reserialize',
 '_to_java_object_rdd',
 'aggregate',
 'aggregateByKey',
 'cache',
 'cartesian',
 'checkpoint',
 'coalesce',
 'cogroup',
 'collect',
 'collectAsMap',
 'combineByKey',
 'context',
 'count',
 'countApprox',
 'countApproxDistinct',
 'countByKey',
 'countByValue',
 'ctx',
 'distinct',
 'filter',
 'first',
 'flatMap',
 'flatMapValues',
 'fold',
 'foldByKey',
 'foreach',
 'foreachPartition',
 'fullOuterJoin',
 'getCheckpointFile',
 'getNumPartitions

In [4]:
# 外部数据源：local file, HDFS, Cassandra,Hbase, S3,包括textFile, SequenceFiles等
# 在HDFS上默认一个block(128mb)一个partition
rdd2 = sc.textFile("file:///E:input/text/p*.txt") # 每个文件一行一条记录
rdd2a = sc.wholeTextFiles("file:///E:input/text/*txt") # pairs[filename, content]

In [5]:
rdd2.count()
#rdd2.coalesce(1).saveAsPickleFile("file:///E:input/text/pickle/rdd2")
rdd2.getNumPartitions()
rdd2_pickle = sc.pickleFile("file:///E:input/text/pickle/rdd2/part-00000",1)

In [6]:
print(rdd2_pickle.collect()[:2])

['name:Mercury\\r\\ndescription:Mercury is the god of commerce...\\r\\ntype:rocky planet', 'name:Venus\\r\\ndescription:Venus is the goddess of love...\\r\\ntype:rocky planet']


In [8]:
rdd2.coalesce(1).saveAsPickleFile("file:///E:input/text/pickle/rdd2")

In [ ]:
'''
输入文件格式
SequenceFiles: flat,binary file type
Parqute:
ORC
JSON
Hive tables
JDBC
Avro
Apache Arrow:


'''

In [ ]:
# 键值对
lines = sc.textFile("file:///E:input/text/test.txt")
import re
#words = lines.flatMap(lambda x : x.split(r"\W+"))
words = lines.flatMap(lambda x : re.split("\W+", x))
word_pairs = words.map(lambda x:(x,1))
counts = word_pairs.reduceByKey(lambda a,b:a+b)
counts.collect()


In [ ]:
'''
Spark转换Transformations:
map(func)
filter(func)
flatMap(func)
mapPartitions(func):类似map,但是按照每个partion(block),func:iterator<T> =>iterator<U>
mapPartitionsWithIndex(func):类似上面带有partition的索引，func: (Int,iterator<T> =>iterator<U>)
sample(withReleacement, fraction,seed): 随机采样
union(otherDataset): 类似sql的union联合两个dataset的元素，不排除重复值
intersection(otherDataset): 返回两个dataset的交集
distinct(numPartitions): 只包含唯一元素的dataset
groupByKey(numPartitions): (k,v) =>(k,iterable<v>), 可以设定任务数量
reduceByKey(func,[numPartitions]): (k,v) => (k,v),func必须是(v,v) =>v, reduce任务的个数可以设定
aggregateByKey(zeroValue)(seqOp,combOp,[numPartitions]): 输入输出的类型不一样(k,v) =>(k,u)，否则用reduceByKey
sortByKey([ascending],[numPartitions]): (k,v)对中k实现Ordered类
sortBy: 全局有序的rdd
join(otherDataset,[numPartitions]): (K,V).join(K,W) =>(K,(V,W)),leftOuterJoin,rightOuterJoin,fullOuterJoin.
cogroup(otherDataset,[numPartitions]): (K,V).join(K,W) =>(K,(Iterable<V>,Iterable<W>)) 
cartesian(otherDataset): 笛卡尔积
pipe(command,[envVars]):rdd的每个partition通过一个shell命令
coalesce(numPartitions):降低partions到指定数量
repartition(numPartitions): 跨过全局重新洗牌创建更多或更少的partition，并且全局balance
repartitionAnsSortWithPartitions(patitioner): 参考partitioner重新分区,在每一个partition内排序，比reparition效率更高
        
'''
aggregateByKey(zeroValue)(seqOp,combOp,[numPartitions])总结：
zeroValue：可以为0如果是求和，" "如果是集合类型, Double.MinValue如果求最大,Doublemax.MaxValue如果求最小，key值不用写
SeqOp转换/合并在同一个partition中的数据，在写func时，key值不用写
CombOp合并不同partition中已经聚集好的数据，在写func时，key值不用写

zero_val = (0,0)
def seq_op(accumulator,element):
def comb_op(accumulator1,accumulator2):
# https://backtobazics.com/big-data/spark/apache-spark-aggregatebykey-example/
'''

In [ ]:
'''
Spark动作Actions
reduce(func): 使用func聚集数据，func要满足交换律和结合律
collect(): 返回全部元素作为一个array到driver程序端，
count(): 返回元素个数
first(): 返回第一个元素
take(n): 返回前n个元素的array
takeSample(withReplacement, num,seed): 返回随机采样的元素集合作为一个array
takeOrdered(n,[ordering]): 返回前n个元素按照自然排序或者自定义排序

saveAsTextFile(path):每个元素调用toString转换，text file格式写到文件系统路径
savaAsSequenceFile(path):仅限java/scala，写dataset的元素作为hadoop的SequenceFile, rdd的k/v对实现hadoop的Writable接口
saveAsObjectFile(path): 使用java serialization写每个元素
countByKey():在(k,v)对的rdd上返回hashmap格式的(k,Int)对

foreach(func): func作用于每个元素，
foreachAsync
'''

In [ ]:
'''
RDD持久化：
MEMEORY_ONLY, MEMORY_AND_DISK,
MEMORY_ONLY_SER: 仅限java/scala, 储存rdd像一个序列化的java object
MEMORY_AND_DISK_SER
'''

In [ ]:
'''
Shared Variables

Broadcast广播变量
broadcast_var = sc.broadcast([1,2,3]) #

Accumulators累加变量
accum = sc.accumulator(0) # Accumulator<id=0,value=0>

'''

In [ ]:
import random
d1 = list(range(15))
d2 = list(range(10,25))
random.shuffle(d1),random.shuffle(d2) #原地洗牌

In [ ]:
d1,d2

In [ ]:
r1 = sc.parallelize(d1)
r2 = sc.parallelize(d2)

In [ ]:
r1.getNumPartitions(),r2.getNumPartitions()

In [ ]:
def func_part(partition): # 输入是每一个partition
    rdd = [] #输出必须为iterator<U>
    for x in partition:
        rdd.append((x,1))
    return rdd # 返回一个Iterator

In [ ]:
r1a = r1.map(lambda x: (x,1)).sortByKey(True,5)
r2a = r2.mapPartitions(func_part).sortByKey()

In [ ]:
r3 = r1a.intersection(r2a).collect()
r4 = r1a.cogroup(r2a)

In [ ]:
r4a =r4.sortByKey().collect()

In [ ]:
it = iter(r4a[10][1][1])
next(it)

In [ ]:
next(it)